In [343]:
import random

# Cards

Let's make a poker game, or at least part of it. Our goal will ultimately be to create a game of 5 card draw poker. For now, we'll try to create a deck of cards, and hands for the players that inherit from the deck. So, we should have 3 classes for this stage: Card, Deck, and Hand:
<ul>
<li> Card</li>
    <ul>
    <li> A card is one individual card, made up of a Suit and a Rank (ace, two, three...queen, etc...)</li>
    <li></li>
    </ul>
<li> Deck</li>
    <ul>
    <li> A deck is made up of a bunch of cards. </li>
    <li></li>
    <li></li>
    </ul>
<li> Hand</li>
    <ul>
    <li> The hand class inheirits from the deck class and represents a "mini-deck" of five cards (we're building a 5 card poker deck for now, later on we'll change this
    up a bit to make this into a five_card_poker deck, and that'll allow us to make other decks that inheirit from the Deck class but make sense for different games).</li>
    <li></li>
    </ul>
</ul>

Underneath the spot we have to make the classes, there is a bunch of stuff to test and see if it works is there. This testing is limited, we should try to create more to really test things here - if we are going to build this up into an actual game of poker (and we intend to), we should make sure that things work first, especially the edge cases.

<b>Note:</b> a full game of poker, especially the part of calculating who wins, is actually reasonably complex. That part isn't really critically important, if we make some error in the calculation of which three of a kind tie-breaker <i>actually</i> wins, it isn't a big deal. The important point is that we can create a deck that works - even if there's some tiny error in the exact logic. We can always fix that later In reality, game logic errors are something we'd notice in user testing, where we would be actually playing to make sure it works, and at some point we'd see that the wrong player won some hand, and we'd have to go back and fix it. This is a bit of agile thinking, right now we're in sprint number one. 

In [344]:
import pandas as pd

class Card():

    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        #if pd.isnull(self.value) or pd.isnull(self.suit):
        #    return ""
        #print("\t\t"+self.suit+" "+self.value)
        return "{} of {}".format(self.rank, self.suit)

    def __repr__(self):
        #print("\t\t"+self.suit+" "+self.value)
        return "{} of {}".format(self.rank, self.suit)

    def __lt__(self, other):
        # check the suits
        if self.suit < other.suit: return True
        if self.suit > other.suit: return False
        # suits are the same... check ranks
        return self.rank < other.rank

    def __gt__(self, other):
        return other.__lt__(self)

    def __eq__(self, other):
        return self.suit == other.suit and self.rank == other.rank

class Deck():

    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    rank = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight',
                'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']
    
    # This stuff is to make a function to convert to short to the short version. 
    # This doesn't really matter function-wise
    short_suits = ['H', 'D', 'C', 'S']
    short_rank = ['2', '3', '4', '5', '6', '7', '8', '9', '10',
                  'J', 'Q', 'K', 'A']
    @staticmethod
    def tooShort(card):
        return Deck.short_rank[Deck.rank.index(card.rank)]+Deck.short_suits[Deck.suits.index(card.suit)]
    
    def __init__(self, *cards, shuffle=True, populate=False):
        self.deck = []
        
        if populate:
            self.populate52()
        elif len(cards) > 0:
            for card in cards:
                self.deck.append(card)

        if shuffle:
            self.shuffle()
    
    def shuffle(self):
        random.shuffle(self.deck)

    def populate52(self):
        self.deck = []
        for suit in self.suits:
            for value in self.rank:
                self.deck.append(Card(suit, value))
    
    def __str__(self):
        return_string = ""
        for i, card in enumerate(self.deck):
            return_string += str(i)+": "+str(card)+"\n"
        return return_string

    def __iter__(self):
        return self

    def __next__(self):
        if self.deck:
            try:
                return self.deck.__next__()
            except:
                raise StopIteration
        else:
            raise StopIteration
    # Fix
    def deal(self, num_hands=1, card_per_hand=1, kind=Deck):
        hands = []
        for i in range(num_hands):
            hand = []
            for j in range(card_per_hand):
                hand.append(self.deck.pop())
            tmp = kind(*hand, shuffle=False, populate=False)
            
            #print(hand)
            #print(tmp)
            
            hands.append(tmp)
        return hands
    
    def addCard(self, card):
        self.deck.append(card)
    def removeCard(self, card):
        for i, c in enumerate(self.deck):
            if c == card:
                return self.deck.pop(i)
    def removeCard(self, suit, rank):
        for i, c in enumerate(self.deck):
            if c == Card(suit, rank):
                return self.deck.pop(i)

class Hand(Deck):
    hands = ['High Card', 'Pair', 'Two Pair', 'Trips', 'Straight', 'Flush',
             'Full House', 'Quads', 'Straight Flush', 'Royal Flush']
    @staticmethod
    def deckToHand(deck):
        return Hand(*deck.deck)

    def __init__(self, *cards, size=5):
        super().__init__(*cards)
        self.size = size
    
    def checkFlush(self):
        suit = self.deck[0].suit
        for card in self.deck:
            if card.suit != suit:
                return False
        return True
    def checkStraight(self):
        rank = self.deck[0].rank
        for card in self.deck:
            if card.rank != rank:
                return False
        return True
    def checkPair(self):
        rank = self.deck[0].rank
        for card in self.deck:
            if card.rank == rank:
                return True
        return False
    def checkTrips(self):
        rank = self.deck[0].rank
        count = 0
        for card in self.deck:
            if card.rank == rank:
                count += 1
        return count == 3
    def checkQuads(self):
        rank = self.deck[0].rank
        count = 0
        for card in self.deck:
            if card.rank == rank:
                count += 1
        return count == 4
    def checkFullHouse(self):
        rank = self.deck[0].rank
        count = 0
        for card in self.deck:
            if card.rank == rank:
                count += 1
        return count == 3
    def checkTwoPair(self):
        rank = self.deck[0].rank
        count = 0
        for card in self.deck:
            if card.rank == rank:
                count += 1
        return count == 2
    def checkStraightFlush(self):
        return self.checkStraight() and self.checkFlush()
    def checkRoyalFlush(self):
        return self.checkStraightFlush() and self.deck[0].rank == 'Ace'
    def checkHand(self):
        if self.checkRoyalFlush():
            return 9
        elif self.checkStraightFlush():
            return 8
        elif self.checkQuads():
            return 7
        elif self.checkFullHouse():
            return 6
        elif self.checkFlush():
            return 5
        elif self.checkStraight():
            return 4
        elif self.checkTrips():
            return 3
        elif self.checkTwoPair():
            return 2
        elif self.checkPair():
            return 1
        else:
            return 0
    def __lt__(self, other):
        if self.checkHand() < other.checkHand():
            return True
        elif self.checkHand() > other.checkHand():
            return False
        else:
            return self.deck[0] < other.deck[0]

## Testing

We should make sure that this works. 

##### Create Deck

Make a new deck and see what's in it. We want one that is ready to play, so we'll want it shuffled and filled with 52 cards. 

In [345]:
d = Deck(shuffle=True, populate=True)
print(d)

0: Five of Hearts
1: Four of Diamonds
2: Four of Clubs
3: Queen of Hearts
4: Ace of Spades
5: Jack of Hearts
6: Eight of Diamonds
7: Five of Spades
8: Nine of Spades
9: Two of Clubs
10: Ten of Clubs
11: Seven of Diamonds
12: Six of Clubs
13: Four of Spades
14: Ten of Spades
15: Three of Diamonds
16: Ten of Diamonds
17: Five of Clubs
18: King of Spades
19: Two of Spades
20: Ace of Hearts
21: Two of Diamonds
22: Three of Hearts
23: Four of Hearts
24: King of Clubs
25: Two of Hearts
26: Five of Diamonds
27: Jack of Diamonds
28: Jack of Clubs
29: Queen of Spades
30: Three of Clubs
31: King of Hearts
32: Jack of Spades
33: Nine of Hearts
34: Nine of Clubs
35: Ace of Clubs
36: Six of Hearts
37: Six of Diamonds
38: Eight of Spades
39: Seven of Spades
40: Seven of Hearts
41: Eight of Hearts
42: Queen of Diamonds
43: Three of Spades
44: Seven of Clubs
45: Eight of Clubs
46: Queen of Clubs
47: Six of Spades
48: Ace of Diamonds
49: King of Diamonds
50: Nine of Diamonds
51: Ten of Hearts



##### Deal Hands, Check Score

Now we should check the dealing of hands - a big one, I checked this a bunch of times while making it, running it over and over. Right now, I'm going to check for a few things:
<ul>
<li> The deal produces the expected results - 4 hands of 5 <i>different</i> cards each. </li>
<li> I'll score each of the cards, based on the score logic defined in the Hand class. </li>
<li> Below this part, I want to make sure that the cards that were dealt into hands here aren't still in the deck. </li>
    <ul>
    <li> We could have built something that tested the deal by making a deck, dealing cards, looking at the cards dealt and left, and verifying programatically that the cards dealt are no longer in the deck. </li>
    <li> <b>Note for Sanity:</b> when we are testing this we need to make sure that which cells we've run is consistent. In mine, we make a deck of 52, then deal from it. If we were to accidentally do extra (or fewer) deals, refresh the deck, or other things before we check the results, we can easily break things. </li>
    </ul>
</ul>

In [346]:
hands = list(map(Hand.deckToHand, d.deal(num_hands=4, card_per_hand=5)))

for hand in hands:
    print(hand.checkHand())
    print(hand)

1
0: Ace of Diamonds
1: Nine of Diamonds
2: Six of Spades
3: King of Diamonds
4: Ten of Hearts

1
0: Three of Spades
1: Eight of Clubs
2: Queen of Clubs
3: Seven of Clubs
4: Queen of Diamonds

2
0: Eight of Spades
1: Seven of Hearts
2: Seven of Spades
3: Eight of Hearts
4: Six of Diamonds

2
0: Nine of Clubs
1: Nine of Hearts
2: Six of Hearts
3: Ace of Clubs
4: Jack of Spades



In [347]:
print(d)

0: Five of Hearts
1: Four of Diamonds
2: Four of Clubs
3: Queen of Hearts
4: Ace of Spades
5: Jack of Hearts
6: Eight of Diamonds
7: Five of Spades
8: Nine of Spades
9: Two of Clubs
10: Ten of Clubs
11: Seven of Diamonds
12: Six of Clubs
13: Four of Spades
14: Ten of Spades
15: Three of Diamonds
16: Ten of Diamonds
17: Five of Clubs
18: King of Spades
19: Two of Spades
20: Ace of Hearts
21: Two of Diamonds
22: Three of Hearts
23: Four of Hearts
24: King of Clubs
25: Two of Hearts
26: Five of Diamonds
27: Jack of Diamonds
28: Jack of Clubs
29: Queen of Spades
30: Three of Clubs
31: King of Hearts



##### Sort and Check Again

Here, we want to sort by the (rough) score of the hand, and see if the hands are in the right order. We can do this by sorting the hands by their score, and then checking the order. This checks the sorting logic, and the scoring logic.

In [348]:
hands.sort(reverse=True)
for hand in hands:
    print(hand.checkHand())
    print(hand)

2
0: Eight of Spades
1: Seven of Hearts
2: Seven of Spades
3: Eight of Hearts
4: Six of Diamonds

2
0: Nine of Clubs
1: Nine of Hearts
2: Six of Hearts
3: Ace of Clubs
4: Jack of Spades

1
0: Three of Spades
1: Eight of Clubs
2: Queen of Clubs
3: Seven of Clubs
4: Queen of Diamonds

1
0: Ace of Diamonds
1: Nine of Diamonds
2: Six of Spades
3: King of Diamonds
4: Ten of Hearts



##### Deal Another Set of Hands

Here, we want to deal another set of hands, and check that the cards are different. We can do this by checking that the cards in the new hands are not in the old hands, and vice versa. We also want to print the deck again, and make sure that it keeps shrinking as we deal from it. 

In [349]:
hands2 = list(map(Hand.deckToHand, d.deal(num_hands=7, card_per_hand=2)))

In [350]:
for hand in hands2:
    print(hand.checkHand())
    print(hand)

1
0: Queen of Spades
1: Jack of Diamonds
2: King of Hearts
3: Three of Clubs
4: Jack of Clubs

1
0: King of Clubs
1: Five of Diamonds
2: Three of Hearts
3: Two of Hearts
4: Four of Hearts

1
0: Five of Clubs
1: King of Spades
2: Two of Spades
3: Ace of Hearts
4: Two of Diamonds

1
0: Three of Diamonds
1: Four of Spades
2: Ten of Spades
3: Six of Clubs
4: Ten of Diamonds



In [351]:
print(d)

0: Five of Hearts
1: Four of Diamonds
2: Four of Clubs
3: Queen of Hearts
4: Ace of Spades
5: Jack of Hearts
6: Eight of Diamonds
7: Five of Spades
8: Nine of Spades
9: Two of Clubs
10: Ten of Clubs
11: Seven of Diamonds



##### Other Card Tests

This is more from development than testing that we still need, but it won't hurt. Here I'm testing the creation of cards and hands manually, as well as the printout of the short form. 

In [352]:
card1 = Card("Hearts", "Queen")
card2 = Card("Spades", "Queen")
card3 = Card("Hearts", "Seven")
card4 = Card("Spades", "Seven")
card5 = Card("Hearts", "Nine")

hand1 = Hand(card1, card2, card3, card4, card5)
print(hand1)

0: Nine of Hearts
1: Queen of Spades
2: Seven of Spades
3: Seven of Hearts
4: Queen of Hearts



In [354]:
print(Deck.tooShort(card1))
print(Deck.tooShort(card3))

QH
7H
